In [1]:
!pip install Keras-Preprocessing
!pip install tensorflow
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
from tensorflow.keras.utils import to_categorical
# Import all the libraries 
import os
import shutil
from PIL import Image, ImageOps  
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('white')
import plotly.graph_objects as go
import plotly.express as px
import datetime
from plotly.offline import init_notebook_mode, iplot
import plotly.io as pio
pio.templates.default = 'plotly_white'
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Concatenate, Flatten, MaxPooling2D, Conv2D
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Add
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras import Model
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import warnings
warnings.filterwarnings("ignore")
print("TF version:", tf.__version__)

TF version: 2.10.0


In [2]:
train_ds = tf.keras.utils.image_dataset_from_directory("C:/Users/TANAY SALVE/Downloads/suyash/Alzheimer_Dataset/train",    
    labels='inferred',
    label_mode='int',
    class_names=None,
    color_mode='rgb',
    batch_size=64,
    image_size=(227, 227),
    shuffle=True,
    seed=42,                                                   
    validation_split=0.2,
    subset="training",                                                  
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
 )

val_ds = tf.keras.utils.image_dataset_from_directory("C:/Users/TANAY SALVE/Downloads/suyash/Alzheimer_Dataset/train",
  validation_split=0.2,
  subset="validation",                                                   
  seed=42,
  image_size=(227, 227),
  batch_size=64)

Found 5121 files belonging to 4 classes.
Using 4097 files for training.
Found 5121 files belonging to 4 classes.
Using 1024 files for validation.


In [3]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

0.0 0.9853845


In [4]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [5]:
import tensorflow as tf

class AttrDict(dict):
    __setattr__ = dict.__setitem__
    __getattr__ = dict.__getitem__


def plot_grid_ds(ds, model=None, size=(3, 3), figsize=(10, 10)):
    from copy import copy
    import matplotlib.pyplot as plt
    from mpl_toolkits.axes_grid1 import ImageGrid
    
    ds_ = copy(ds)
    n = size[0] * size[1]
    
    if isinstance(ds_, tf.keras.preprocessing.image.DirectoryIterator):
        class_names = list(ds.class_indices.keys())
        iterator = list(next(ds_))
        iterator = tuple([i[:n] for i in iterator])
        X_batch, y_batch = iterator
        y_batch = y_batch.astype(int)
    else:
        class_names = ds_.class_names
        iterator = ds_.unbatch().shuffle(n).batch(n).take(1)
        X_batch, y_batch = next(iterator)

    fig = plt.figure(figsize=figsize)
    grid = ImageGrid(
        fig,
        111,
        nrows_ncols=size,
        axes_pad=0.3,
    )
    y_hat_batch = model.predict(X_batch).argmax(axis=-1) if model else y_batch
    for X, y, y_hat, ax in zip(X_batch, y_batch, y_hat_batch, grid):
        title = (
            f"{class_names[y]} (true) - {class_names[y_hat]} (pred)"
            if model
            else f"{class_names[y]}"
        )
        ax.set_title(title)
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)
        ax.imshow(X, cmap="gray")
    return grid


class BalancedSparseCategoricalAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):
    def __init__(self, name="balanced_sparse_categorical_accuracy", dtype=None):
        super().__init__(name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_flat = y_true
        if y_true.shape.ndims == y_pred.shape.ndims:
            y_flat = tf.squeeze(y_flat, axis=[-1])
        y_true_int = tf.cast(y_flat, tf.int32)

        cls_counts = tf.math.bincount(y_true_int)
        cls_counts = tf.math.reciprocal_no_nan(tf.cast(cls_counts, self.dtype))
        weight = tf.gather(cls_counts, y_true_int)
        return super().update_state(y_true, y_pred, sample_weight=weight)

class Patches(tf.keras.layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


class PatchEncoder(tf.keras.layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = tf.keras.layers.Dense(units=projection_dim)
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


class ViTClassfier(tf.keras.models.Model):
    CONFIG = {
        "patch_size": 6,
        "embedding_dim": 32,
        "n_heads": 2,
        "n_transformers": 1,
        "transformer_units": [64, 32],
        "transformer_dropout": 0.1,
        "mlp_units": [2048, 1024],
        "mlp_dropout": 0.1,
    }

    def __init__(self, input_shape, n_classes, config={}):
        super(ViTClassfier, self).__init__()

        self._config = AttrDict(
            **{
                **self.CONFIG,
                **config,
                "input_shape": input_shape,
                "n_classes": n_classes,
            }
        )
        self._construct_model()

    def call(self, x):
        return self._model(x)

    def _construct_model(self):
        config = self._config

        num_patches = (config.input_shape[0] // config.patch_size) ** 2

        inputs = tf.keras.layers.Input(shape=config.input_shape)

        patches = Patches(config.patch_size)(inputs)
        # Encode patches.
        encoded_patches = PatchEncoder(num_patches, config.embedding_dim)(patches)

        # Create multipletf.keras.layers of the Transformer block.
        for _ in range(config.n_transformers):
            # Layer normalization 1.
            x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
            # Create a multi-head attention layer.
            attention_output = tf.keras.layers.MultiHeadAttention(
                num_heads=config.n_heads, key_dim=config.embedding_dim, dropout=0.1
            )(x1, x1)
            # Skip connection 1.
            x2 = tf.keras.layers.Add()([attention_output, encoded_patches])
            # Layer normalization 2.
            x3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x2)
            # MLP.
            for units in config.transformer_units:
                x3 = tf.keras.layers.Dense(units, activation=tf.nn.gelu)(x3)
                x3 = tf.keras.layers.Dropout(config.transformer_dropout)(x3)

            # Skip connection 2.
            encoded_patches = tf.keras.layers.Add()([x3, x2])

        # Create a [batch_size, projection_dim] tensor.
        features = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        features = tf.keras.layers.Flatten()(features)
        features = tf.keras.layers.Dropout(0.5)(features)
        # Add MLP.
        for units in config.mlp_units:
            features = tf.keras.layers.Dense(units, activation=tf.nn.gelu)(features)
            features = tf.keras.layers.Dropout(config.mlp_dropout)(features)
        # Classify outputs.
        logits = tf.keras.layers.Dense(config.n_classes)(features)
        # Create the Keras model.
        self._model = tf.keras.models.Model(inputs=inputs, outputs=logits)

In [6]:
hparams = AttrDict(
    image_size=(64, 64, 3),
    n_classes=4,
    batch_size=64,
)

ds = tf.keras.preprocessing.image_dataset_from_directory("C:/Users/TANAY SALVE/Downloads/suyash/Alzheimer_Dataset/train", image_size=hparams.image_size[:-1], batch_size=hparams.batch_size)
train_split = round(0.8 * ds.cardinality().numpy()) 
train_ds = ds.take(train_split); train_ds.class_names = ds.class_names
validation_ds = ds.skip(train_split); validation_ds.class_names = ds.class_names

Found 5121 files belonging to 4 classes.


In [7]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = ViTClassfier(hparams.image_size, hparams.n_classes)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy", BalancedSparseCategoricalAccuracy()]
    )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [13]:
model.fit(train_ds, epochs=100, validation_data=validation_ds)

Epoch 1/100
65/65 [==============================] - 19s 284ms/step - loss: 0.1770 - accuracy: 0.9305 - balanced_sparse_categorical_accuracy: 0.9379 - val_loss: 0.0432 - val_accuracy: 0.9906 - val_balanced_sparse_categorical_accuracy: 0.9891
Epoch 2/100
65/65 [==============================] - 19s 295ms/step - loss: 0.1480 - accuracy: 0.9447 - balanced_sparse_categorical_accuracy: 0.9524 - val_loss: 0.0575 - val_accuracy: 0.9886 - val_balanced_sparse_categorical_accuracy: 0.9892
Epoch 3/100
65/65 [==============================] - 19s 292ms/step - loss: 0.1308 - accuracy: 0.9517 - balanced_sparse_categorical_accuracy: 0.9505 - val_loss: 0.0464 - val_accuracy: 0.9886 - val_balanced_sparse_categorical_accuracy: 0.9788
Epoch 4/100
65/65 [==============================] - 21s 322ms/step - loss: 0.1182 - accuracy: 0.9553 - balanced_sparse_categorical_accuracy: 0.9530 - val_loss: 0.0498 - val_accuracy: 0.9844 - val_balanced_sparse_categorical_accuracy: 0.9842
Epoch 5/100
65/65 [=============

Epoch 35/100
65/65 [==============================] - 18s 278ms/step - loss: 0.0266 - accuracy: 0.9909 - balanced_sparse_categorical_accuracy: 0.9876 - val_loss: 0.0020 - val_accuracy: 0.9990 - val_balanced_sparse_categorical_accuracy: 0.9994
Epoch 36/100
65/65 [==============================] - 18s 277ms/step - loss: 0.0502 - accuracy: 0.9839 - balanced_sparse_categorical_accuracy: 0.9785 - val_loss: 0.0054 - val_accuracy: 0.9990 - val_balanced_sparse_categorical_accuracy: 0.9993
Epoch 37/100
65/65 [==============================] - 18s 280ms/step - loss: 0.0272 - accuracy: 0.9906 - balanced_sparse_categorical_accuracy: 0.9907 - val_loss: 5.9537e-04 - val_accuracy: 1.0000 - val_balanced_sparse_categorical_accuracy: 1.0000
Epoch 38/100
65/65 [==============================] - 18s 277ms/step - loss: 0.0243 - accuracy: 0.9911 - balanced_sparse_categorical_accuracy: 0.9918 - val_loss: 8.4568e-04 - val_accuracy: 1.0000 - val_balanced_sparse_categorical_accuracy: 1.0000
Epoch 39/100
65/65 [

Epoch 69/100
65/65 [==============================] - 18s 274ms/step - loss: 0.0124 - accuracy: 0.9952 - balanced_sparse_categorical_accuracy: 0.9960 - val_loss: 0.0019 - val_accuracy: 1.0000 - val_balanced_sparse_categorical_accuracy: 1.0000
Epoch 70/100
65/65 [==============================] - 18s 272ms/step - loss: 0.0216 - accuracy: 0.9923 - balanced_sparse_categorical_accuracy: 0.9917 - val_loss: 0.0013 - val_accuracy: 1.0000 - val_balanced_sparse_categorical_accuracy: 1.0000
Epoch 71/100
65/65 [==============================] - 18s 274ms/step - loss: 0.0170 - accuracy: 0.9930 - balanced_sparse_categorical_accuracy: 0.9840 - val_loss: 0.0022 - val_accuracy: 0.9990 - val_balanced_sparse_categorical_accuracy: 0.9962
Epoch 72/100
65/65 [==============================] - 18s 274ms/step - loss: 0.0298 - accuracy: 0.9925 - balanced_sparse_categorical_accuracy: 0.9861 - val_loss: 0.0212 - val_accuracy: 0.9938 - val_balanced_sparse_categorical_accuracy: 0.9940
Epoch 73/100
65/65 [========

In [14]:
import numpy as np
from sklearn.metrics import confusion_matrix

X, y_true = list(zip(*[(X, y) for (X, y) in validation_ds.unbatch().as_numpy_iterator()]))
y_hat = model.predict_on_batch(np.array(X)).argmax(axis=-1)
mat = confusion_matrix(y_true, y_hat)
mat

array([[132,   0,   0,   0],
       [  0,   8,   0,   0],
       [  0,   0, 496,   0],
       [  0,   0,   0, 325]], dtype=int64)

In [15]:
from sklearn.metrics import balanced_accuracy_score

acc = balanced_accuracy_score(y_true, y_hat)
print(f"Final accuracy on validation set : {acc:.3f}")

Final accuracy on validation set : 1.000


In [16]:
model.save('path_to_my_model',save_format='tf')

INFO:tensorflow:Assets written to: path_to_my_model\assets


INFO:tensorflow:Assets written to: path_to_my_model\assets


In [17]:
model.save("vits.model")

INFO:tensorflow:Assets written to: vits.model\assets


INFO:tensorflow:Assets written to: vits.model\assets
